In [1]:
import pandas as pd
import numpy as np
from numpy import random
import gensim
from gensim import corpora
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
import string
from nltk.tokenize.toktok import ToktokTokenizer

import glob
from joblib import Parallel, delayed
import os
from tqdm import tqdm

import itertools
import multiprocessing
from itertools import islice

cpu_count = multiprocessing.cpu_count()

DATA_FOLDER = '../data/'

In [2]:
# all_docs = glob.iglob(os.path.join(DATA_FOLDER, 'FIPS') + '/**/*.txt', recursive=True)
# corpus_file = open(DATA_FOLDER + 'corpus.txt', 'w')
# for item in all_docs:
#     corpus_file.write("%s\n" % os.path.relpath(item, DATA_FOLDER))

In [3]:
# from nltk.tokenize.toktok import ToktokTokenizer 
# toktok = ToktokTokenizer()
# toktok.tokenize('устройство по п . 3 , отличать тем , что оно снабжать устанавливать в головка винт , в который размещать шарик и ролик .  . ')

In [3]:
stop_list = stopwords.words('russian')
stop_list.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на', 'ко'])
stop_list.extend(list(string.punctuation))

punkts = [s for s in string.punctuation if s not in '.!?']


class Corpus(gensim.corpora.TextCorpus):
    def get_texts(self):
        for filename in tqdm(self.input): # for each relevant file
            yield tokenize(open(filename).read())

            
def save_corpus(list_block, dir_name, prefix):
    corpus = Corpus(list_block)
    
    dic_name = os.path.join(dir_name, '%s.dict' % prefix)
    corp_name = os.path.join(dir_name, '%s_corpus.mm' % prefix)
    
    corpus.dictionary.save(dic_name)
    corpora.MmCorpus.serialize(corp_name, corpus)
    
    return dic_name, corp_name
    
            
def tokenize(file_text):
    tokens = nltk.word_tokenize(file_text)

    tokens = [validate_word(word) for word in tokens if check_word(word)]
    tokens = filter(lambda s: s != '', tokens)
    
    return tokens

            
def validate_word(word):
    word = word.replace("«", "").replace("»", "").replace("`", '').replace(".", '')
    if len(word) == 1:
        return ''

    return word


def check_word(word):
    if word in stop_list:
        return False
    if any(char.isdigit() for char in word):
        return False
    
    return True

In [4]:
def grouper(n, iterable):
    it = iter(iterable)
    while True:
       chunk = tuple(islice(it, n))
       if not chunk:
           return
       yield chunk

In [ ]:
%%time
# dfolder = os.path.join(DATA_FOLDER, 'FIPS/Inventions patents_txt_output/0c/0c')
# all_docs = glob.glob(dfolder + '/**/*.txt', recursive=True)

all_docs = glob.glob(os.path.join(DATA_FOLDER, 'FIPS') + '/**/*.txt', recursive=True)

parallelizer = Parallel(n_jobs=cpu_count)

# this iterator returns the functions to execute for each task
tasks_iterator = ( delayed(save_corpus)(list_block, os.path.join(DATA_FOLDER, 'BoW'), i) for 
                  i, list_block in enumerate(grouper(len(all_docs)//100, all_docs)) ) 
result = parallelizer( tasks_iterator )

  0%|          | 16/11944 [00:01<18:13, 10.91it/s]]s]

In [8]:
merged_corpus = None
for dic_name, corp_name in result:
    cur_dict = corpora.Dictionary.load(dic_name)
    cur_corp = corpora.MmCorpus(corp_name)
    
    if not merged_corpus:
        dict1 = cur_dict
        merged_corpus = cur_corp
        continue

    cur_to_dict1 = dict1.merge_with(cur_dict)
    merged_corpus = itertools.chain(merged_corpus, cur_to_dict1[cur_corp])

dict1.save(os.path.join(DATA_FOLDER, 'dict.dict'))
corpora.MmCorpus.serialize(os.path.join(DATA_FOLDER, 'corpus.mm'), merged_corpus)